## 🧪 E2: Best Model 기준 Precision으로 변경 실험

> **목표**: 베스트 모델 선택 기준을 **F1 → Precision**으로 변경하여 오탐(FP) 감소

### 실험 내용
- `metric_for_best_model`을 `precision`으로 변경
- Precision이 가장 높은 epoch의 모델을 최종 모델로 선택
- F1 대비 Precision 향상, Recall 하락 예상

### 변경 사항
- `metric_for_best_model`: **f1 → precision** ⭐
- `class_weights`: [1, 1] (기존 유지)
- `threshold`: 0.5 (기존 유지)

In [1]:
# ========================================
# 📦 패키지 설치 (필요시)
# ========================================
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
%pip install transformers==4.42.0 datasets accelerate scikit-learn pandas matplotlib seaborn -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ========================================
# 🔧 경로 설정 (본인 환경에 맞게 수정)
# ========================================
import os

# 절대 경로 사용 (본인 환경에 맞게 수정하세요)
DATA_DIR = r"c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data"
OUTPUT_DIR = r"./E2_output"

# 경로 확인
if not os.path.exists(DATA_DIR):
    print(f"⚠️ 데이터 폴더를 찾을 수 없습니다: {DATA_DIR}")
    print(f"   현재 작업 디렉토리: {os.getcwd()}")
else:
    print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 데이터 폴더: c:\00_Sookmyunguniv\DACOS(2025)\2학기\프로젝트\github\25-2-team3\data
📁 출력 폴더: ./E2_output


In [3]:
# ========================================
# 1) 환경 준비
# ========================================
import os
import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import (precision_recall_fscore_support, accuracy_score, 
                             roc_auc_score, confusion_matrix)
from datasets import Dataset, DatasetDict
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          Trainer, TrainingArguments)

# 시드 고정
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    print(f"✅ GPU 사용 가능: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU가 감지되지 않았습니다. CPU로 실행됩니다.")

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

✅ GPU 사용 가능: NVIDIA GeForce RTX 4060 Ti


In [4]:
# ========================================
# 2) 데이터 로드
# ========================================
abusive_path = os.path.join(DATA_DIR, "combined_abusive_shuffled_20k.csv")
nonabusive_path = os.path.join(DATA_DIR, "nonabusive_merged_shuffled_sample20000.csv")

df_ab = pd.read_csv(abusive_path)[["text", "label"]]
df_non = pd.read_csv(nonabusive_path)[["text", "label"]]

df = pd.concat([df_ab, df_non], ignore_index=True).dropna().reset_index(drop=True)

print(f"📊 라벨 분포:\n{df['label'].value_counts()}")
print(f"\n총 데이터 수: {len(df)}개")

📊 라벨 분포:
label
0    22882
1    17118
Name: count, dtype: int64

총 데이터 수: 40000개


In [5]:
# ========================================
# 3) Train/Test 분리
# ========================================
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["label"]
)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

Train: 32000, Test: 8000


In [6]:
# ========================================
# 4) 토큰화
# ========================================
MODEL_ID = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_fn(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

raw_ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(test_df),
})

tokenized = raw_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format("torch")

print("✅ 토큰화 완료!")

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

✅ 토큰화 완료!


In [7]:
# ========================================
# 5) 클래스 가중치 (기존과 동일)
# ========================================
labels_np = train_df["label"].values
pos = (labels_np == 1).sum()
neg = (labels_np == 0).sum()
w0 = 1 / neg
w1 = 1 / pos
scale = 2 / (w0 + w1)
class_weights = torch.tensor([w0 * scale, w1 * scale], dtype=torch.float)
print(f"Class weights: {class_weights}")

Class weights: tensor([0.8559, 1.1441])


In [8]:
# ========================================
# 6) 모델 + Trainer 설정
# ⭐ metric_for_best_model = "precision"
# ========================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", zero_division=0)
    
    # Confusion matrix for FP
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = float("nan")
    
    return {
        "accuracy": acc, 
        "precision": prec, 
        "recall": rec, 
        "f1": f1, 
        "roc_auc": auc,
        "FP": fp,
        "FPR": fpr,
    }

base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID, num_labels=2)

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ⭐ 핵심 변경: metric_for_best_model = "precision"
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="precision",  # ⭐ F1 → Precision
    greater_is_better=True,
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=base_model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("✅ Trainer 설정 완료!")
print("⭐ metric_for_best_model = 'precision'")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Trainer 설정 완료!
⭐ metric_for_best_model = 'precision'


In [9]:
# ========================================
# 7) 학습 실행
# ========================================
trainer.train()

  0%|          | 0/3000 [00:00<?, ?it/s]

{'loss': 0.6839, 'grad_norm': 2.425734519958496, 'learning_rate': 4.9e-06, 'epoch': 0.05}
{'loss': 0.5756, 'grad_norm': 4.3658647537231445, 'learning_rate': 9.9e-06, 'epoch': 0.1}
{'loss': 0.4256, 'grad_norm': 4.2172532081604, 'learning_rate': 1.47e-05, 'epoch': 0.15}
{'loss': 0.33, 'grad_norm': 2.977836847305298, 'learning_rate': 1.9699999999999998e-05, 'epoch': 0.2}
{'loss': 0.2531, 'grad_norm': 6.806024551391602, 'learning_rate': 2.47e-05, 'epoch': 0.25}
{'loss': 0.2753, 'grad_norm': 5.396688461303711, 'learning_rate': 2.97e-05, 'epoch': 0.3}
{'loss': 0.2796, 'grad_norm': 3.0265049934387207, 'learning_rate': 2.9477777777777776e-05, 'epoch': 0.35}
{'loss': 0.2442, 'grad_norm': 4.198986530303955, 'learning_rate': 2.8922222222222224e-05, 'epoch': 0.4}
{'loss': 0.2438, 'grad_norm': 6.425168991088867, 'learning_rate': 2.8366666666666667e-05, 'epoch': 0.45}
{'loss': 0.2502, 'grad_norm': 1.1815909147262573, 'learning_rate': 2.781111111111111e-05, 'epoch': 0.5}
{'loss': 0.2291, 'grad_norm':

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.1905481219291687, 'eval_accuracy': 0.92275, 'eval_precision': 0.9190561529271206, 'eval_recall': 0.8986565420560748, 'eval_f1': 0.9087418783225044, 'eval_roc_auc': 0.9789849506874552, 'eval_FP': 271, 'eval_FPR': 0.059222027972027975, 'eval_runtime': 16.6895, 'eval_samples_per_second': 479.344, 'eval_steps_per_second': 14.979, 'epoch': 1.0}
{'loss': 0.14, 'grad_norm': 3.675835371017456, 'learning_rate': 2.1711111111111112e-05, 'epoch': 1.05}
{'loss': 0.1326, 'grad_norm': 3.6549899578094482, 'learning_rate': 2.116666666666667e-05, 'epoch': 1.1}
{'loss': 0.1524, 'grad_norm': 2.2765233516693115, 'learning_rate': 2.0611111111111112e-05, 'epoch': 1.15}
{'loss': 0.1476, 'grad_norm': 6.985886096954346, 'learning_rate': 2.0055555555555556e-05, 'epoch': 1.2}
{'loss': 0.1488, 'grad_norm': 3.094674587249756, 'learning_rate': 1.95e-05, 'epoch': 1.25}
{'loss': 0.1359, 'grad_norm': 4.245350360870361, 'learning_rate': 1.8944444444444447e-05, 'epoch': 1.3}
{'loss': 0.145, 'grad_norm': 1

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.20958544313907623, 'eval_accuracy': 0.92825, 'eval_precision': 0.9320800485142511, 'eval_recall': 0.8977803738317757, 'eval_f1': 0.9146087473966081, 'eval_roc_auc': 0.9809351078973597, 'eval_FP': 224, 'eval_FPR': 0.04895104895104895, 'eval_runtime': 16.6953, 'eval_samples_per_second': 479.178, 'eval_steps_per_second': 14.974, 'epoch': 2.0}
{'loss': 0.0682, 'grad_norm': 4.678597927093506, 'learning_rate': 1.0611111111111111e-05, 'epoch': 2.05}
{'loss': 0.0723, 'grad_norm': 6.029900074005127, 'learning_rate': 1.0055555555555557e-05, 'epoch': 2.1}
{'loss': 0.0735, 'grad_norm': 2.24930477142334, 'learning_rate': 9.5e-06, 'epoch': 2.15}
{'loss': 0.0812, 'grad_norm': 3.4027059078216553, 'learning_rate': 8.944444444444444e-06, 'epoch': 2.2}
{'loss': 0.0845, 'grad_norm': 6.12247371673584, 'learning_rate': 8.388888888888888e-06, 'epoch': 2.25}
{'loss': 0.065, 'grad_norm': 12.429498672485352, 'learning_rate': 7.833333333333333e-06, 'epoch': 2.3}
{'loss': 0.0599, 'grad_norm': 1.74

  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.28701451420783997, 'eval_accuracy': 0.927375, 'eval_precision': 0.9321982365460626, 'eval_recall': 0.8954439252336449, 'eval_f1': 0.913451511991658, 'eval_roc_auc': 0.9806868666161526, 'eval_FP': 223, 'eval_FPR': 0.04873251748251748, 'eval_runtime': 16.644, 'eval_samples_per_second': 480.654, 'eval_steps_per_second': 15.02, 'epoch': 3.0}
{'train_runtime': 982.7408, 'train_samples_per_second': 97.686, 'train_steps_per_second': 3.053, 'train_loss': 0.1650975329875946, 'epoch': 3.0}


TrainOutput(global_step=3000, training_loss=0.1650975329875946, metrics={'train_runtime': 982.7408, 'train_samples_per_second': 97.686, 'train_steps_per_second': 3.053, 'total_flos': 6314665328640000.0, 'train_loss': 0.1650975329875946, 'epoch': 3.0})

In [10]:
# ========================================
# 8) 최종 평가
# ========================================
final_metrics = trainer.evaluate()

print("\n" + "="*60)
print("📊 E2 최종 평가 결과 (metric=precision)")
print("="*60)
for key, value in final_metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

  0%|          | 0/250 [00:00<?, ?it/s]


📊 E2 최종 평가 결과 (metric=precision)
  eval_loss: 0.2870
  eval_accuracy: 0.9274
  eval_precision: 0.9322
  eval_recall: 0.8954
  eval_f1: 0.9135
  eval_roc_auc: 0.9807
  eval_FP: 223
  eval_FPR: 0.0487
  eval_runtime: 16.6774
  eval_samples_per_second: 479.6900
  eval_steps_per_second: 14.9900
  epoch: 3.0000


In [12]:
# ========================================
# 📝 9) 실험 결과 기록
# ========================================
print("\n" + "="*60)
print("📝 실험 결과 기록 (복사용)")
print("="*60)

# 변수 미리 포맷팅 (f-string 내 조건식+포맷 에러 방지)
fpr_str = f"{final_metrics['eval_FPR']:.4f}" if isinstance(final_metrics.get('eval_FPR'), float) else 'N/A'
prec_str = f"{final_metrics['eval_precision']:.4f}" if isinstance(final_metrics.get('eval_precision'), float) else 'N/A'
rec_str = f"{final_metrics['eval_recall']:.4f}" if isinstance(final_metrics.get('eval_recall'), float) else 'N/A'
f1_str = f"{final_metrics['eval_f1']:.4f}" if isinstance(final_metrics.get('eval_f1'), float) else 'N/A'
auc_str = f"{final_metrics['eval_roc_auc']:.4f}" if isinstance(final_metrics.get('eval_roc_auc'), float) else 'N/A'

print(f"""
#### 실험 ID: E2
- 변경 사항:
  - metric_for_best_model: f1 → precision
  - class weight: 기존 유지 [{class_weights[0]:.4f}, {class_weights[1]:.4f}]
  - threshold: 0.5 (기존 유지)
- 데이터:
  - train/validation split: 0.8/0.2 (random_state=42, stratify=label)
  - max_length=128, batch_size=32, epochs=3
- Validation 성능:
  - FP(정상→악성): {final_metrics.get('eval_FP', 'N/A')}
  - FPR: {fpr_str}
  - Precision: {prec_str}
  - Recall: {rec_str}
  - F1: {f1_str}
  - AUC: {auc_str}
- 코멘트:
  - Precision 기준으로 best model 선택
  - F1 기준 대비 Precision ↑, Recall ↓ 예상
""")


📝 실험 결과 기록 (복사용)

#### 실험 ID: E2
- 변경 사항:
  - metric_for_best_model: f1 → precision
  - class weight: 기존 유지 [0.8559, 1.1441]
  - threshold: 0.5 (기존 유지)
- 데이터:
  - train/validation split: 0.8/0.2 (random_state=42, stratify=label)
  - max_length=128, batch_size=32, epochs=3
- Validation 성능:
  - FP(정상→악성): 223
  - FPR: 0.0487
  - Precision: 0.9322
  - Recall: 0.8954
  - F1: 0.9135
  - AUC: 0.9807
- 코멘트:
  - Precision 기준으로 best model 선택
  - F1 기준 대비 Precision ↑, Recall ↓ 예상



In [13]:
# ========================================
# 💾 10) 모델 저장
# ========================================
SAVE_DIR = os.path.join(OUTPUT_DIR, "best_model")
os.makedirs(SAVE_DIR, exist_ok=True)
trainer.save_model(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

print(f"✅ 모델 저장 완료: {SAVE_DIR}")

✅ 모델 저장 완료: ./E2_output\best_model
